* code by Sihyun You (2021.12.28.)
* edit by Jehyun Lee (2021.12.30.)
* revised for mrnIF by Jehyun Lee (2022.01.08.)

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pybliometrics.scopus import ScopusSearch
from bs4 import BeautifulSoup
import requests, json
from docx import Document
from copy import deepcopy

In [2]:
# Scopus API Keys
from my_apikeys import APIKeys

In [3]:
# subscriber (Institute: True. Home: False)
try:
    s_sample = ScopusSearch(f"DOI (10.1038/s41598-021-83315-9)").results[0]
    subscriber=True
except:
    subscriber=False

print(f"subscriber={subscriber}")    

subscriber=True


In [4]:
def regularize_date_publication(_str):
    _str = _str.replace(",", "")
    token_date = _str.split(' ')
    if len(token_date) == 1: # year only
        year = _str
        s = _str
    elif (re.match('[0-9]', token_date[0])):
        day, month, year = token_date[0], token_date[1][:3].upper(), token_date[2]
        s = ' '.join([month, day, year])
    elif (re.match('[A-Za-z]', token_date[0]) and re.match('[0-9]', token_date[1]) and int(token_date[1])<32):    
        month, day, year = token_date[0][:3].upper(), token_date[1], token_date[2]
        s = ' '.join([month, day, year])
    else:
        month, year = token_date[0][:3].upper(), token_date[1]
        s = ' '.join([month, year])
        
    return (int(year), s)

def get_pub_index(_pub_name, _df):
    list_title = _df['TITLE'].map(regularize)
    pub_index = np.where(regularize(_pub_name) == list_title)[0]
    if len(pub_index) > 0:
        return pub_index
    return np.array([])

def get_pub_index_eissn(_pub_eissn, _df):
    list_eissn = _df['EISSN'].str.replace("-","").values
    pub_index = np.where(_pub_eissn == list_eissn)[0]
    if len(pub_index) > 0:
        return pub_index
    return np.array([])

def regularize(_str):
    return re.sub('[^A-Za-z0-9]+', '', re.sub('&', 'and', _str)).lower()       

def regularize_space(_str):
    return re.sub('[^A-Za-z0-9]+', ' ', re.sub('&', 'and', _str)).lower()       

In [5]:
pd.set_option("mode.chained_assignment", None)
pd.set_option("display.max_columns", None)
dfs_JCR_SCIE = {}
YEAR_START, YEAR_REMARK, YEAR_THIS = 2016, 2020, 2021

In [6]:
for y in range(YEAR_START, YEAR_THIS):
    print(f"{y}년도 시트를 로딩중입니다.")
    dfs_JCR_SCIE.update({str(y):pd.read_excel("./data/JCR_SCIE_(2016-2020)_merged.xlsx", sheet_name=f"JCR {y}")})

2016년도 시트를 로딩중입니다.
2017년도 시트를 로딩중입니다.
2018년도 시트를 로딩중입니다.
2019년도 시트를 로딩중입니다.
2020년도 시트를 로딩중입니다.


In [7]:
# EISSN 인가
for k in dfs_JCR_SCIE.keys():
    col_capitals = np.array([c.upper() for c in dfs_JCR_SCIE[k].columns])
    if "EISSN" in col_capitals:
        idx_eissn = np.where("EISSN" == col_capitals)[0][0]
        dfs_JCR_SCIE[k] = dfs_JCR_SCIE[k].rename(columns={dfs_JCR_SCIE[k].columns[idx_eissn]:"EISSN"})

EISSN_2019 = dfs_JCR_SCIE["2019"][["Title20", "ISO_ABBREV", "TITLE", "ISSN", "EISSN"]]
EISSN_2020 = dfs_JCR_SCIE["2020"][["Title20", "ISO_ABBREV", "TITLE", "ISSN", "EISSN"]]
EISSN = pd.concat([EISSN_2019, EISSN_2020], axis=0).drop_duplicates("ISSN")

for k in dfs_JCR_SCIE.keys():
    col_capitals = np.array([c.upper() for c in dfs_JCR_SCIE[k].columns])
    if "EISSN" not in col_capitals:
        dfs_JCR_SCIE[k] = dfs_JCR_SCIE[k].merge(EISSN, how="left", on=["Title20", "ISO_ABBREV", "TITLE", "ISSN"])

In [8]:
dfs_JCR_SCIE.keys()

dict_keys(['2016', '2017', '2018', '2019', '2020'])

In [9]:
df_applicants = pd.read_excel("./data/HR_input_from949.xlsx", header=1, dtype={"UT":str})

# display example
df_applicants.head(3)

,전체순번,수험번호내 순번,수험번호,이름,영문명,논문제목,게재일자,지원자 입력 DOI,수정 DOI,SCIE구분,역할,게재지명,출판사,ISSN,논문구분\n(SCIE),Publication Date,#citation,Publication Year journal impact factor,2020\njournal\nimpact\nfactor,2020 journal impact factor percentile,1st Author,1ST AUTHOR\n(Y/N),Reprint Author,REPRINT AUTHOR\n(Y/N),Source\n(Journal),volume,issue,Notes
0,877,2,0088-000162,박재현,park jae hyun,Designing an ultrathin silica layer for highly...,2014.08.06,10.1039/C4NR04293J,10.1039/C4NR04293J,국외SCIE,주저자,"Nanoscale, 6, 20, 12111-12119, 2014",Royal society of chemistry,2040-3372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1008,5,0088-000184,박준용,"Park, Junyong",Enhancement of dye-sensitized solar cells usin...,2014.01.29,10.1039/c4ra00194j,10.1039/c4ra00194j,국외SCIE,주저자,"RSC Advances., 2014, 4, 9946",Royal Society of Chemistry,2046-2069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,107,9,0088-000021,이민오,"Lee, Minoh",Freeze-dried MoS2 sponge electrodes for enhanc...,2017.02.21,10.1039/C6DT04466B,10.1039/C6DT04466B,국외SCIE,기타,"Dalton Transactions, 46, 7, 2122-2128, 2017",Royal Society of Chemistry,1477-9226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
list_title = df_applicants["논문제목"].values
list_name_kr = df_applicants["이름"].values
list_name_en = df_applicants["영문명"].values
list_doi = df_applicants["수정 DOI"].values

In [11]:
print(len(list_doi), len(list_title))

581 581


In [12]:
df_applicants.columns

Index(['전체순번', '수험번호내 순번', '수험번호', '이름', '영문명', '논문제목', '게재일자', '지원자 입력 DOI',
       '수정 DOI', 'SCIE구분', '역할', '게재지명', '출판사', 'ISSN', '논문구분\n(SCIE)',
       'Publication Date', '#citation',
       'Publication Year journal impact factor',
       '2020\njournal\nimpact\nfactor',
       '2020 journal impact factor percentile', '1st Author',
       '1ST AUTHOR\n(Y/N)', 'Reprint Author', 'REPRINT AUTHOR\n(Y/N)',
       'Source\n(Journal)', 'volume', 'issue', 'Notes'],
      dtype='object')

In [19]:
info_scopus

Document(eid='2-s2.0-85013345100', doi='10.1039/c6dt04466b', pii=None, pubmed_id='28119985', title='Freeze-dried MoS<inf>2</inf> sponge electrodes for enhanced electrochemical energy storage', subtype='ar', subtypeDescription='Article', creator='Balasingam S.K.', afid='60103153;60025960;60013141;60012291;60000142', affilname='Ulsan National Institute of Science and Technology;Korea Institute of Science and Technology;Norges Teknisk-Naturvitenskapelige Universitet;Incheon National University;Konkuk University', affiliation_city='Ulsan;Seoul;Trondheim;Incheon;Seoul', affiliation_country='South Korea;South Korea;Norway;South Korea;South Korea', author_count='5', author_names='Balasingam, Suresh Kannan;Lee, Minoh;Kim, Byung Hoon;Lee, Jae Sung;Jun, Yongseok', author_ids='55658056717;55572233000;57209098631;26643592500;7102042662', author_afids='60000142-60013141;60103153-60025960;60012291;60103153;60000142', coverDate='2017-01-01', coverDisplayDate='2017', publicationName='Dalton Transactio

In [26]:
xmls = []

for i, (doi, title) in enumerate(zip(list_doi, list_title)):
    s = ScopusSearch(f"DOI ({doi})", download=True, subscriber=subscriber).results
    if s == None:
        try:
            s = ScopusSearch(f"TITLE ({title})", download=True, subscriber=subscriber).results
            if s != None:
                doi = s[0].doi
                df_applicants["Notes"][i] = f"DOI 오류: {doi}"
            else:
                print(f"- No.{i} is invalid.")
                df_applicants["Notes"][i] = "doi 및 title 확인 필요"
                continue
        except:
            print(f"- No.{i} is not accessible in Scopus.")
            df_applicants["Notes"][i] = "Scopus에서 접근 불가. 확인 필요"
            continue
            
        
    info_scopus = s[0]

    index_remark = get_pub_index(info_scopus.publicationName, dfs_JCR_SCIE[str(YEAR_REMARK)])
    if len(index_remark) == 0:
        index_remark = get_pub_index_eissn(info_scopus.eIssn, dfs_JCR_SCIE[str(YEAR_REMARK)])

    scie_yn = 'N'
    if len(index_remark) > 0:
        scie_yn = 'Y'
    else:
        scie_yn = 'N'
        print("SCIE 논문이 아닙니다.")
        continue
    
    df_applicants["논문구분\n(SCIE)"][i] = scie_yn
    
    year, date = regularize_date_publication(info_scopus.coverDisplayDate)
    df_applicants["Publication Date"][i] = date
    df_applicants["#citation"][i] = str(info_scopus.citedby_count)
    
    if year < YEAR_START:
        df_applicants["Notes"][i] = f"{YEAR_START-1} 이전 논문"
        continue
    
    elif year < YEAR_THIS:
        list_n = get_pub_index(info_scopus.publicationName, dfs_JCR_SCIE[str(year)])
        if len(list_n) > 0:
            index_n = list_n[0]
        else:
            list_n = get_pub_index_eissn(info_scopus.eIssn, dfs_JCR_SCIE[str(year)])
            if len(list_n) > 0:
                index_n = list_n[0]
            else:
                print("IF를 찾지 못했습니다.")
                continue
        jif_n = str(dfs_JCR_SCIE[str(year)]["IMPACT_FACTOR"][index_n])
    else:
        jif_n = f"{YEAR_THIS}년 출판으로 IF미발급"
        
    df_applicants["Publication Year journal impact factor"][i] = jif_n
    df_applicants["2020\njournal\nimpact\nfactor"][i] = str(dfs_JCR_SCIE[str(YEAR_REMARK)]["IMPACT_FACTOR"][index_remark[0]])
    
    ifp = dfs_JCR_SCIE[str(YEAR_REMARK)].loc[index_remark, "mrnIF"]
    try: 
        df_applicants["2020 journal impact factor percentile"][i] = str(ifp.values[0])
    except:
        df_applicants["2020 journal impact factor percentile"][i] = "mrnIF 없음"
    
    ### Author data
    authors_raw = np.array([regularize(n) for n in info_scopus.author_names.split(";")])
    
    list_name_eni = list_name_en[i]
    if "," not in list_name_eni: # 이름 성 
        list_name_eni_ = deepcopy(list_name_eni)
        list_name_eni_ = list_name_eni_.split(" ")
        list_name_eni = ", ".join(list_name_eni_[1:] + list_name_eni_[:1])
        
    try:
        idx_author = np.where(regularize(list_name_eni) == authors_raw)[0][0]
    except IndexError: # swap family and last name
        try:
            idx_author = np.where(regularize("".join(list_name_eni.split(",")[::-1])) == authors_raw)[0][0]
        except IndexError : # possibly disturbed by middle names
            idx_author=None
            author_name_words = regularize_space(list_name_eni).split(" ")
            name_words = [regularize_space(name).rstrip(" ").split(" ") for name in info_scopus.author_names.split(";")]
            for j, name_word in enumerate(name_words):
                name_check = list(set(author_name_words) - set(name_word))
                if len(name_check) == 0:
                    idx_author = j
                    df_applicants["Notes"][i] = "지원자 성명 확인 필요"
                    
    if idx_author == None:
        # 이니셜로만 되어있는건 아닌지 확인
        familyname = list_name_eni.split(", ")[0].lower()
        firstname = list_name_eni.split(", ")[1].lower()
        name_TF = [True if ((n[0] == familyname and len(set(n[1:])-set(firstname))==0) or 
                            (n[-1]==familyname and len(set(n[:-1])-set(firstname))==0)) 
                   else False 
                   for n in name_words]
        if len(np.where(np.array(name_TF)==True)[0]) > 0:
            idx_author = np.where(np.array(name_TF)==True)[0][0]
            df_applicants["Notes"][i] = "논문 저자 이름이 약어로 표현됨. 확인 필요."
            continue
        else:
            df_applicants["Notes"][i] = "지원자가 저자 명단에 없음"
            
        
    # first author    
    if idx_author == 0:
        first_author = list_name_eni
        first_author_yn = "Y"
    else:
        first_author = info_scopus.author_names.split(";")[0]
        first_author_yn = "N"    
    
    df_applicants["1st Author"][i] = first_author
    df_applicants["1ST AUTHOR\n(Y/N)"][i] = first_author_yn    
    df_applicants["1st Author"][i] = first_author
    
        
    ### Publication
    df_applicants["Source\n(Journal)"][i] = info_scopus.publicationName.upper()
    df_applicants["volume"][i] = info_scopus.volume
    
    if info_scopus.issueIdentifier != None:
        issue = info_scopus.issueIdentifier
    else:
        issue = ''
    df_applicants["issue"][i] = issue
    
    # PDF download
    accept = "application/pdf"
    HEADERS = {
        'X-ELS-APIKEY': APIKeys[-1],
        'Accept': accept
    }
    url = f'http://api.elsevier.com/content/article/doi:{doi}?view=FULL'
    try:
        r = requests.get(url, stream=True, headers=HEADERS)
        if r.status_code == 200: # download supported in Scopus
            if accept == "application/pdf":
                for chunk in r.iter_content(chunk_size=1024*1024):
                    with open(f"./pdf/paper_{i+950}.pdf", "wb") as f:
                        f.write(chunk)
            elif accept == "text/xml":
                xml = BeautifulSoup(r.content, "html5lib")
                xmls.append(xml)
                with open(f"./xml/xml_{i+950}.xml", "w") as f:
                    f.write(xml.prettify())

        elif r.status_code == 404: # not supported in Scopus
            print(f"# No.{i} is not available in Scopus.")
            
        # XML data mining
        url = f"https://doi.org/{doi}"
        r = requests.get(url)
        xml = BeautifulSoup(r.content, "html5lib")
        xmls.append(xml)            
        with open(f"./xml/xml_{i+950}.xml", "w") as f:
            f.write(xml.prettify())
    except:
        pass

    # write in every step
    df_applicants.to_excel("applicants_fill_from949.xlsx", index=False)
    
print(len(xmls))


# No.2 is not available in Scopus.
# No.3 is not available in Scopus.
# No.4 is not available in Scopus.
# No.5 is not available in Scopus.
- No.7 is not accessible in Scopus.
# No.8 is not available in Scopus.
# No.9 is not available in Scopus.
# No.10 is not available in Scopus.
# No.12 is not available in Scopus.
# No.13 is not available in Scopus.
# No.14 is not available in Scopus.
# No.15 is not available in Scopus.
# No.16 is not available in Scopus.
# No.17 is not available in Scopus.
# No.18 is not available in Scopus.
# No.19 is not available in Scopus.
# No.20 is not available in Scopus.
# No.21 is not available in Scopus.
# No.22 is not available in Scopus.
# No.23 is not available in Scopus.
# No.24 is not available in Scopus.
# No.25 is not available in Scopus.
# No.26 is not available in Scopus.
# No.27 is not available in Scopus.
# No.28 is not available in Scopus.
# No.29 is not available in Scopus.
# No.30 is not available in Scopus.
# No.31 is not available in Scopus

In [17]:
doi

'10.1039/C6DT04466B'

In [23]:
title

'Freeze-dried MoS2 sponge electrodes for enhanced electrochemical energy storage'

In [ ]:
list_doi[i]

In [ ]:
year

In [ ]:
dfs_JCR_SCIE[str(year)]["ISSN"]

In [ ]:
info_scopus.issn

In [ ]:
info_scopus.publicationName

In [ ]:
dfs_JCR_SCIE[str(year)]["EISSN"]

In [18]:
doi = "10.1039/C6DT04466B"
s = ScopusSearch(f"DOI ({doi})", download=True, subscriber=subscriber).results
info_scopus = s[0]
index_remark_eissn = get_pub_index_eissn(info_scopus.eIssn, dfs_JCR_SCIE[str(YEAR_REMARK)])
print(index_remark_eissn)
index_remark = get_pub_index(info_scopus.publicationName, dfs_JCR_SCIE[str(YEAR_REMARK)])
print(index_remark)

[6302]
[6302]


In [ ]:
info_scopus = s[0]

index_remark = get_pub_index(info_scopus.publicationName, dfs_JCR_SCIE[str(YEAR_REMARK)])
if len(index_remark) == 0:
    index_remark = get_pub_index_eissn(info_scopus.eIssn, dfs_JCR_SCIE[str(YEAR_REMARK)])

In [ ]:
index_remark

In [ ]:
title

In [ ]:
info_scopus.publicationName

In [ ]:
"Progress in Photovoltaics".lower() in [v.lower() for v in dfs_JCR_SCIE[str(YEAR_REMARK)]["TITLE"].values]

In [ ]:
s

In [ ]:
info_scopus.eIssn

In [ ]:
info_scopus.eIssn in dfs_JCR_SCIE[str(YEAR_REMARK)]["EISSN"].str.replace("-","").values

In [ ]:
dfs_JCR_SCIE[str(YEAR_REMARK)]["EISSN"]

In [ ]:
df_A = pd.DataFrame({"X":[1,2,3,4,5], "Y":[11,12,13,14,15]})
df_B = pd.DataFrame({"X":[1,2,3,5,6], "Y":[11,12,13,15,16]})
df_C = pd.DataFrame({"X":[2,3,5,6,7], "Y":[12,13,15,16,17], "Z":[22,23,25,26,27]})

In [ ]:
df_A.merge(df_C, how="left")

In [ ]:
dfs_JCR_SCIE["2018"].head(3)

In [ ]:
dfs_JCR_SCIE["2017"].head(3)

In [ ]:
dfs_JCR_SCIE["2020"].head(3)